## Preparar el entorno y carga del modelo 

In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification

# Mapea índice -> categoría (lo que usaste en training)
index_to_category = {
    0: "INDUSTRIAL", 
    1: "CIVIL", 
    2: "INFORMÁTICA",
    3: "TELECOMUNICACIONES",
    4: "ARQUITECTURA",
    5: "FILOSOFÍA",
    6: "PSICOLOGÍA",
    7: "LETRAS",
    8: "COMUNICACIÓN SOCIAL",
    9: "EDUCACIÓN",
    10: "ADMINISTRACIÓN",
    11: "CONTADURÍA",
    12: "RELACIONES INDUSTRIALES",
    13: "SOCIOLOGÍA",
    14: "ECONOMÍA",
    15: "DERECHO",
    16: "TEOLOGÍA"
}


# Dispositivo (GPU si está disponible)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Cargar tokenizer
tokenizer = BertTokenizer.from_pretrained('dccuchile/bert-base-spanish-wwm-cased')

# Cargar modelo con la misma configuración
model = BertForSequenceClassification.from_pretrained(
    'dccuchile/bert-base-spanish-wwm-cased',
    num_labels=len(index_to_category)
)
model.load_state_dict(torch.load('best_model_state.bin', map_location=device))
model.to(device)
model.eval()  # Modo inferencia

In [ ]:
## predict carrer

def predict_career(text_list, tokenizer, model, max_length=128):
    """
    text_list: lista de strings con las respuestas de usuario
    tokenizer: tokenizer de Hugging Face
    model: modelo BertForSequenceClassification cargado
    """
    # Tokenizar
    inputs = tokenizer(
        text_list,
        padding=True,
        truncation=True,
        max_length=max_length,
        return_tensors="pt"
    )
    
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)

    model.eval()
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits  # [batch_size, num_labels]
    
    # Obtener predicciones
    preds = torch.argmax(logits, dim=1).cpu().numpy()  # Índices de clase
    predicted_labels = [index_to_category[p] for p in preds]
    return predicted_labels




import torch

def predict_career_top3(text_list, tokenizer, model, index_to_category, 
                        device, max_length=128, top_k=3, threshold=0.5):
    """
    text_list: lista de strings con las respuestas de usuario
    tokenizer: tokenizer de Hugging Face
    model: modelo BertForSequenceClassification cargado
    index_to_category: diccionario {índice: categoría}
    device: 'cuda' o 'cpu'
    max_length: longitud máxima de tokenización
    top_k: número de predicciones (top) a retornar
    threshold: umbral mínimo de probabilidad para asignar una clase (vs 'UNKNOWN')
    """
    # Tokenizar
    inputs = tokenizer(
        text_list,
        padding=True,
        truncation=True,
        max_length=max_length,
        return_tensors="pt"
    )
    
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)

    model.eval()
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits  # [batch_size, num_labels]

        # Convertir logits a probabilidades con softmax
        probs = torch.softmax(logits, dim=1)  # [batch_size, num_labels]
        
        # Obtener los top_k índices y probabilidades
        top_probs, top_indices = probs.topk(top_k, dim=1, largest=True, sorted=True)
        # top_probs, top_indices => ([batch_size, top_k]), ([batch_size, top_k])

    batch_results = []
    for i in range(len(text_list)):
        # Extraer las k probabilidades e índices para esta fila i
        row_probs = top_probs[i].cpu().numpy()
        row_indices = top_indices[i].cpu().numpy()

        result = []
        for idx, p in zip(row_indices, row_probs):
            category_name = index_to_category[idx]
            result.append((category_name, float(p)))
        batch_results.append(result)

    return batch_results

In [ ]:
# Texto filtrado y concatenado
user_text = """
Me gustan el inglés, biología y psicología porque me sentía conectada con esas materias. 
Por el contrario, física, química y matemática se me hicieron difíciles. 
He realizado cursos de fotografía, oratoria y modelos de naciones unidas.

Disfruto correr, pintar, dormir, pasear y ver películas. Me interesan la cultura, el arte, los derechos humanos y los temas sociales. 
Aprendería más sobre periodismo de investigación y documentación, porque considero fundamental contar aquello que se mantiene oculto. 
Me motiva investigar, ayudar y enseñar.

Me interesa entender cómo funciona el mundo, prefiero resolver problemas prácticos y concretos, 
disfruto trabajar con personas y ayudarlas, y me apasiona explorar culturas, ideas o filosofías. 
Visualizo un estudio creativo o colaborar con una ONG que promueva proyectos sociales.

Mi meta es terminar mis dos carreras y fundar una ONG que ayude y enseñe a mujeres y niños en situaciones complejas.
"""



top3_preds = predict_career_top3(
    text_list=[user_text],
    tokenizer=tokenizer,
    model=model,
    index_to_category=index_to_category,
    device=device,
    top_k=3,
    threshold=0.5
)
print("Top 3 predicciones:", top3_preds[0])